In [ ]:
!pip install pandas numpy yfinance matplotlib seaborn ipywidgets

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import VBox, HBox, Text, Button, Output, widgets, GridBox, Dropdown
from IPython.display import display

In [ ]:
#To make this interactive, I'm creating the text boxes for tickers, investment amount and a dropdown for the period range for analysis.
#I'm using range(3) here because I assume that there are 3 stocks in the user's portfolio. However, I've included an "Add More Inputs" button incase it's more than 3

ticker_inputs = [widgets.Text(description=f'Ticker {i+1}:', placeholder='E.g., AAPL') for i in range(3)]
investment_inputs = [widgets.Text(description=f'Investment {i+1}:', placeholder='E.g., 1000') for i in range(3)]
period_input = widgets.Dropdown(
    options=[('1 Month', '1mo'), ('3 Months', '3mo'), ('6 Months', '6mo'), ('1 Year', '1y'), ('3 Years', '3y')],
    value='1y',
    description='Period:'
)
add_button = widgets.Button(description='Add More Inputs', button_style='info')
submit_button = widgets.Button(description='Analyze Portfolio', button_style='success')
output = Output()


#Now, I'm going to define what the "Add More Input" button is going to do and also to ensure that it appends to the existing text boxes

def add_input_fields(_):
    new_ticker_input = Text(description=f'Ticker {len(ticker_inputs) + 1}:', placeholder='E.g., AAPL')
    new_investment_input = Text(description=f'Investment {len(investment_inputs) + 1}:', placeholder='E.g., 1000')
    ticker_inputs.append(new_ticker_input)
    investment_inputs.append(new_investment_input)

    ui.children = [
        VBox([HBox([ticker_inputs[i], investment_inputs[i]]) for i in range(len(ticker_inputs))]),
        period_input,
        HBox([add_button, submit_button]),
        output
    ]


#Now, I'm going to create a pipeline to Yahoo Finance. This will be our source database. Data from Yahoo Finance comes with "Close", "Open", "Volume", "High", "Low", "Adj Close"
#However, we will only need the adjusted close data for this

def fetch_data(tickers, period):
    data = yf.download(tickers, period=period)['Adj Close']
    return data

def analyze_portfolio(tickers, investments, period):
    data = fetch_data(tickers, period)

    returns = data.pct_change().dropna()

    total_investment = sum(investments)
    weights = [inv / total_investment for inv in investments]

    portfolio_return = np.dot(returns.mean(), weights) * 252
    portfolio_volatility = np.sqrt(np.dot(weights, np.dot(returns.cov() * 252, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility

    correlation_matrix = returns.corr()

    with output:
        output.clear_output()
        print("\nPortfolio Analysis Results:")
        print(f"Expected Annual Return: {portfolio_return:.2%}")
        print(f"Portfolio Volatility: {portfolio_volatility:.2%}")
        print(f"Sharpe Ratio: {sharpe_ratio:.2f}\n")

#Here, I'm basically plotting the graph showing the distribution of stocks to total portfolio, the correlation matrix and the risk assessment graph as well as the proposed recommendations
        plt.figure(figsize=(10, 5))
        plt.pie(weights, labels=tickers, autopct='%1.1f%%', startangle=140)
        plt.title("Portfolio Allocation")
        plt.show()

        plt.figure(figsize=(10, 6))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        plt.title("Correlation Matrix")
        plt.show()

        asset_returns = returns.mean() * 252
        asset_volatility = returns.std() * np.sqrt(252)

        plt.figure(figsize=(10, 6))
        plt.scatter(asset_volatility, asset_returns, c='blue', label='Individual Assets')
        plt.scatter(portfolio_volatility, portfolio_return, c='red', label='Portfolio', marker='X', s=100)
        plt.xlabel('Volatility (Standard Deviation)')
        plt.ylabel('Return')
        plt.title('Risk vs Return')
        plt.legend()
        plt.grid()
        plt.show()

        print("Recommendations:")
        high_corr_assets = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1) > 0)
        high_corr_pairs = high_corr_assets.stack().nlargest(3)

        for pair, corr in high_corr_pairs.items():
            print(f"- Assets {pair[0]} and {pair[1]} are highly correlated (Correlation: {corr:.2f}). Consider diversifying further.")

def on_submit(_):
    tickers = [t.value.strip() for t in ticker_inputs if t.value.strip()]
    investments = []
    try:
        investments = [float(i.value.strip()) for i in investment_inputs if i.value.strip()]
    except ValueError:
        with output:
            output.clear_output()
            print("Error: Ensure all investment inputs are numeric.")
        return

    period = period_input.value
    if len(tickers) != len(investments):
        with output:
            output.clear_output()
            print("Error: The number of tickers and investments must match.")
    else:
        analyze_portfolio(tickers, investments, period)

submit_button.on_click(on_submit)

submit_button.on_click(on_submit)
add_button.on_click(add_input_fields)

ui = VBox([
    VBox([HBox([ticker_inputs[i], investment_inputs[i]]) for i in range(len(ticker_inputs))]),
    period_input,
    HBox([add_button, submit_button]),
    output
])

display(ui)

[*********************100%***********************]  6 of 6 completed
